In [1]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



print(os.cpu_count())

# Optional: Allow TensorFlow to dynamically adjust the number of threads
#tf.config.threading.set_intra_op_parallelism_threads(32)
#tf.config.threading.set_inter_op_parallelism_threads()

print("Intra-op parallelism threads:", tf.config.threading.get_intra_op_parallelism_threads())
print("Inter-op parallelism threads:", tf.config.threading.get_inter_op_parallelism_threads())

2024-11-08 23:39:28.188386: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 23:39:28.188803: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-08 23:39:28.191023: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-08 23:39:28.197154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731105568.208164 1431109 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731105568.21

32
Intra-op parallelism threads: 0
Inter-op parallelism threads: 0


2024-11-08 23:39:29.377798: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [2]:
sequence_length = 10 # Length of the sequence to be fed into the model
num_targets = 4  # Number of targets in your data
max_future_target = 400  # The maximum future timestep to predict
train_instances_per_vessel=1000

In [ ]:
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")

In [ ]:
vessel_data = pd.read_csv("../Datasets/vessels.csv", delimiter="|")
port_data = pd.read_csv("../Datasets/ports.csv", delimiter="|")

In [5]:

# Define the categorical and numerical columns
categorical_columns = ["vesselId", "shippingLineId"]
numerical_columns = ["CEU", "DWT", "GT", "breadth", "draft", "length"]

# Fill NaN values with the mean for numerical columns
vessel_data[numerical_columns] = vessel_data[numerical_columns].fillna(vessel_data[numerical_columns].mean())

# Scale only the numerical columns
vessel_normalizer = MinMaxScaler()
vessel_data[numerical_columns] = vessel_normalizer.fit_transform(vessel_data[numerical_columns])

# Initialize encoders
vessel_encoder = LabelEncoder()
shipping_encoder = LabelEncoder()

vessel_encoder.fit(vessel_data["vesselId"])
shipping_encoder.fit(vessel_data["shippingLineId"])


# Now `vessel_data` contains scaled numerical values and unchanged categorical IDs


LabelEncoder()

In [6]:
train_data["time"] = pd.to_datetime(train_data["time"])
test_data["time"] = pd.to_datetime(test_data["time"])

In [7]:
train_data = train_data.merge(
    vessel_data[["vesselId", "shippingLineId"]], on="vesselId", how="left"
)

In [8]:
port_data_renamed = pd.DataFrame()
port_data_renamed[["portId", "port_latitude", "port_longitude"]] = port_data[
    ["portId", "latitude", "longitude"]
]
train_data = train_data.merge(port_data_renamed, on="portId", how="left")

In [9]:
print(train_data.columns)

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'etaRaw', 'latitude',
       'longitude', 'vesselId', 'portId', 'shippingLineId', 'port_latitude',
       'port_longitude'],
      dtype='object')


In [10]:
def group_navstat(navstat_original):
    """
    Groups the original 16 navstat codes into 5 categories.
    
    Args:
        navstat_original (np.ndarray): Original navstat codes, shape (num_samples, sequence_length)
    
    Returns:
        np.ndarray: Grouped navstat codes, shape (num_samples, sequence_length)
    """
    # Define mapping from original codes to grouped categories
    mapping = {
        0: 0,  # Underway
        1: 1,  # Stationary
        2: 1,  # Stationary
        3: 1,  # Stationary
        4: 1,  # Stationary
        5: 1,  # Stationary
        6: 1,  # Stationary
        7: 2,  # Fishing
        8: 0,  # Underway
        9: 3,  # Other
        10: 3, # Other
        11: 3, # Other
        12: 3, # Other
        13: 3, # Other
        14: 4, # Emergency
        15: 4  # Emergency
    }
    
    # Vectorized mapping
    vectorized_mapping = np.vectorize(mapping.get)
    navstat_grouped = vectorized_mapping(navstat_original)
    
    return navstat_grouped

In [11]:
train_data_preprocessed = train_data

train_data_preprocessed.loc[train_data_preprocessed["cog"] >= 360, "cog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["sog"] >= 1023, "sog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["rot"] == -128, "rot"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["heading"] == 511, "heading"] = (
    np.nan
)



pattern = r"^\d{2}-\d{2} \d{2}:\d{2}$"
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].where(
    train_data_preprocessed["etaRaw"].str.match(pattern, na=False), np.nan
)


train_data_preprocessed = train_data_preprocessed.sort_values("time")

print(train_data_preprocessed.head())


train_data_preprocessed = (
    train_data_preprocessed.groupby("vesselId")
    .apply(lambda group: group.ffill().bfill())
    .reset_index(drop=True)
)


print(train_data_preprocessed.head())

train_data_preprocessed["heading"] = train_data_preprocessed["heading"].fillna(0)

train_data_preprocessed = train_data_preprocessed.dropna().reset_index(drop=True)


# Replace '00-' in etaRaw with the corresponding month and day from the 'time' column
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("00-", na=False),
    "01" + train_data_preprocessed["etaRaw"].str[2:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("-00", na=False),
    train_data_preprocessed["etaRaw"].str[:2]
    + "-01"
    + train_data_preprocessed["etaRaw"].str[5:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains(":60", na=False),
    train_data_preprocessed["etaRaw"].str[:9] + "59",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("60:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "01:00",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("24:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "23:59",
)


train_data_preprocessed["etaRaw"] = pd.to_datetime(
    train_data_preprocessed["time"].dt.year.astype(str)
    + "-"
    + train_data_preprocessed["etaRaw"]
    + ":00",
    format="%Y-%m-%d %H:%M:%S",
)


train_data_preprocessed["seconds_to_eta"] = (
    train_data_preprocessed["etaRaw"] - train_data_preprocessed["time"]
).dt.total_seconds()

train_data_preprocessed = train_data_preprocessed.drop(columns=["etaRaw"])

                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-01 00:00:25  284.0   0.7  0.0     88.0        0  01-09 23:00   
1 2024-01-01 00:00:36  109.6   0.0 -6.0    347.0        1  12-29 20:00   
2 2024-01-01 00:01:45  111.0  11.0  0.0    112.0        0  01-02 09:00   
3 2024-01-01 00:03:11   96.4   0.0  0.0    142.0        1  12-31 20:00   
4 2024-01-01 00:03:51  214.0  19.7  0.0    215.0        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  \
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689   
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19   
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126   
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3   

             shippingLineId  port_latitude  port_longitude  
0  61ec65aea8cafc0e93f0e9

/tmp/ipykernel_1431109/3926903193.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.ffill().bfill())


                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-12 14:07:47  308.1  17.1 -6.0    316.0        0  01-08 06:00   
1 2024-01-12 14:31:00  307.6  17.3  5.0    313.0        0  01-14 23:30   
2 2024-01-12 14:57:23  306.8  16.9  5.0    312.0        0  01-14 23:30   
3 2024-01-12 15:18:48  307.9  16.9  6.0    313.0        0  01-14 23:30   
4 2024-01-12 15:39:47  307.0  16.3  7.0    313.0        0  01-14 23:30   

   latitude  longitude                  vesselId                    portId  \
0   7.50361   77.58340  61e9f38eb937134a3c4bfd8b  61d376b393c6feb83e5eb50c   
1   7.57302   77.49505  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546   
2   7.65043   77.39404  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546   
3   7.71275   77.31394  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546   
4   7.77191   77.23585  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546   

             shippingLineId  port_latitude  port_longitude  
0  61a8e672f9cba188601e84

In [ ]:
train_data_engineered = train_data_preprocessed
train_latitude_radians = np.deg2rad(train_data_engineered["latitude"])
train_longitude_radians = np.deg2rad(train_data_engineered["longitude"])
train_cog_radians = np.deg2rad(train_data_engineered["cog"])
train_heading_radians = np.deg2rad(train_data_engineered["heading"])

port_latitude_radians = np.deg2rad(train_data_engineered["port_latitude"])
port_longitude_radians = np.deg2rad(train_data_engineered["port_longitude"])

train_hour = np.deg2rad(train_data_engineered["time"].dt.hour * 360 / 24)
train_day = np.deg2rad(train_data_engineered["time"].dt.day * 360 / 30)
train_month = np.deg2rad(train_data_engineered["time"].dt.month * 360 / 12)


train_latitude_sin = np.sin(train_latitude_radians)
train_latitude_cos = np.cos(train_latitude_radians)
train_longitude_sin = np.sin(train_longitude_radians)
train_longitude_cos = np.cos(train_longitude_radians)

port_latitude_sin = np.sin(port_latitude_radians)
port_latitude_cos = np.cos(port_latitude_radians)
port_longitude_sin = np.sin(port_longitude_radians)
port_longitude_cos = np.cos(port_longitude_radians)

train_cog_sin = np.sin(train_cog_radians)
train_cog_cos = np.cos(train_cog_radians)

train_heading_sin = np.sin(train_heading_radians)
train_heading_cos = np.cos(train_heading_radians)

train_hour_sin = np.sin(train_hour)
train_hour_cos = np.cos(train_hour)

train_day_sin = np.sin(train_day)
train_day_cos = np.cos(train_day)

train_month_sin = np.sin(train_month)
train_month_cos = np.cos(train_month)


train_data_engineered["latitude_sin"] = train_latitude_sin
train_data_engineered["latitude_cos"] = train_latitude_cos
train_data_engineered["longitude_sin"] = train_longitude_sin
train_data_engineered["longitude_cos"] = train_longitude_cos
train_data_engineered["port_latitude_sin"] = port_latitude_sin
train_data_engineered["port_latitude_cos"] = port_latitude_cos
train_data_engineered["port_longitude_sin"] = port_longitude_sin
train_data_engineered["port_longitude_cos"] = port_longitude_cos
train_data_engineered["cog_sin"] = train_cog_sin
train_data_engineered["cog_cos"] = train_cog_cos
train_data_engineered["heading_sin"] = train_heading_sin
train_data_engineered["heading_cos"] = train_heading_cos

train_data_engineered["hour_sin"] = train_hour_sin
train_data_engineered["hour_cos"] = train_hour_cos
train_data_engineered["day_sin"] = train_day_sin
train_data_engineered["day_cos"] = train_day_cos
train_data_engineered["month_sin"] = train_month_sin
train_data_engineered["month_cos"] = train_month_cos


train_data_engineered["cog_sog_sin"] = (
    train_data_engineered["cog_sin"] * train_data_engineered["sog"]
)
train_data_engineered["cog_sog_cos"] = (
    train_data_engineered["cog_cos"] * train_data_engineered["sog"]
)

train_data_engineered = train_data_engineered.drop(
    columns=[
        "latitude",
        "longitude",
        "cog",
        "heading",
        "portId",
        "cog_sin",
        "cog_cos",
        "sog",
        #"shippingLineId", 
        "navstat",
        "heading_sin",
        "heading_cos",
        "port_latitude",
        "port_longitude",
        #"port_latitude_sin",
        #"port_latitude_cos",
        #"port_longitude_sin",
        #"port_longitude_cos",
        "hour_sin",
        "hour_cos",
        "day_sin",
        "day_cos",
        "month_sin",
        "month_cos",
        "rot", #probably not important, removed because I needed only 18 features
    ],
    axis=1,
)
print(train_data_engineered.columns)

seconds_to_eta_scaler = MinMaxScaler()
rot_scaler = MinMaxScaler()
cog_sog_sin_scaler = MinMaxScaler()
cog_sog_cos_scaler = MinMaxScaler()

train_data_engineered["seconds_to_eta"] = seconds_to_eta_scaler.fit_transform(
    train_data_engineered["seconds_to_eta"].values.reshape(-1, 1)
)
train_data_engineered["cog_sog_sin"] = cog_sog_sin_scaler.fit_transform(
    train_data_engineered["cog_sog_sin"].values.reshape(-1, 1)
)
train_data_engineered["cog_sog_cos"] = cog_sog_cos_scaler.fit_transform(
    train_data_engineered["cog_sog_cos"].values.reshape(-1, 1)
)



Index(['time', 'vesselId', 'shippingLineId', 'seconds_to_eta', 'latitude_sin',
       'latitude_cos', 'longitude_sin', 'longitude_cos', 'port_latitude_sin',
       'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos',
       'cog_sog_sin', 'cog_sog_cos'],
      dtype='object')


In [13]:
# Specify the columns you want to move to the end
columns_to_move_last = ["latitude_sin", "latitude_cos" ,"longitude_sin","longitude_cos"]

# Get a list of all columns
all_columns = train_data_engineered.columns.tolist()

# Filter out the columns you want to move to the end
columns_to_keep = [col for col in all_columns if col not in columns_to_move_last]

# Create the new column order: first keep all other columns, then add the specified columns at the end
new_column_order = columns_to_keep + columns_to_move_last

# Reorder the DataFrame
train_data_engineered = train_data_engineered[new_column_order]

# Check the column order
print(train_data_engineered.columns)

Index(['time', 'vesselId', 'shippingLineId', 'seconds_to_eta',
       'port_latitude_sin', 'port_latitude_cos', 'port_longitude_sin',
       'port_longitude_cos', 'cog_sog_sin', 'cog_sog_cos', 'latitude_sin',
       'latitude_cos', 'longitude_sin', 'longitude_cos'],
      dtype='object')


In [14]:

num_features=len(train_data_engineered.columns)-3 +1 # remove time vesselId and shippingLaneId, add time_diff
print(num_features)

12


In [16]:
time_diff_scaler = MinMaxScaler()
time_diff_index = 0


def create_sequences(
    data: pd.DataFrame,
    sequence_length: int,
    train_instances_per_vessel: int,
    min_sequence_length: int = 10,
    max_future_target=400,
):
    """
    Creates sequences of a specified length from the input data for each vessel.

    Args:
        data (pd.DataFrame): The input data containing 'vesselId', 'time', and feature columns.
        sequence_length (int): The desired length of each sequence.
        max_future_target (int): The maximum number of steps ahead to predict.
        train_instances_per_vessel (int): The number of training instances to generate per vessel.
        min_sequence_length (int): The minimum number of data points required to create a sequence.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, int]:
            A tuple containing the input sequences (X), targets (y), vessel IDs, times, and time_diff_index.
    """
    sequences = []
    targets = []
    vessel_ids = []
    shipping_ids=[]
    times = []
    all_time_diffs = []

    

    # Combine non_sequential_feature_columns with other columns to exclude them from sequential_feature_columns
    sequential_feature_columns = [
        col for col in data.columns if col not in ["vesselId","time", "shippingLineId"]
    ]

    sequential_feature_columns.append("time_diff")

    print("Feature Columns:", sequential_feature_columns)

    # Group data by 'vesselId'
    grouped = data.groupby("vesselId")

    future_target=1

    while future_target<=max_future_target:

        for vessel_id, group in grouped:
            # Sort the group by 'time'
            group = group.sort_values("time").reset_index(drop=True)
            shipping_id=group["shippingLineId"][0]
    
            # Skip vessels with insufficient data
            if len(group) < min_sequence_length:
                continue
    
            # Calculate time differences to the next instance
            group["time_diff"] = (
                (group["time"].diff(-1).dt.total_seconds()/10**6).abs().fillna(0)
            )
            
    
            # Convert features to numpy array
            feature_array = group[sequential_feature_columns].values
    
            # Pad sequences if necessary
            if len(feature_array) < sequence_length + future_target:
                padding_length = sequence_length + future_target - len(feature_array)
                padding = np.zeros((padding_length, feature_array.shape[1]))
                feature_array = np.vstack([feature_array, padding])
    
            # Update the group length after padding
            group_length = len(feature_array)
    
            num_possible_sequences = group_length - sequence_length - int(future_target/2) + 1
    
            # Determine the actual number of sequences we can generate
            actual_instances = min(train_instances_per_vessel, num_possible_sequences)
            if actual_instances <= 0:
                continue
    
            # Calculate step size
            if actual_instances == 1:
                step_size = 0  # Only one sequence possible
            else:
                step_size = num_possible_sequences / (actual_instances) - 1
    
            for i in range(actual_instances):
                k = int(i * step_size)
                seq_x = feature_array[k : k + sequence_length].copy()
    
                # Randomly select future target N steps ahead (1 to future_target inclusive)
                N = future_target
                
                target_idx = k + sequence_length + N  # Adjust index accordingly
    
                # Handle the case where target_idx is beyond the length of the data
                if target_idx >= group_length:
                    # setting last known position as target (in order to use more of the data for longer future targets)
                    target_idx = group_length-1
    
                # Get seq_y
                seq_y = feature_array[target_idx,-5:-1]  # Assuming target variables are included
    
                last_seq_time = group["time"].iloc[min(k + sequence_length - 1, len(group)-1)]
                target_time = group["time"].iloc[min(target_idx, len(group)-1)]
                time_diff_cumulative = (target_time - last_seq_time).total_seconds()/10**6
    
                # Update the last time_diff in seq_x to be time_diff_cumulative
                seq_x[-1, -1] = time_diff_cumulative
    
                all_time_diffs.extend(seq_x[:, -1])
    
                # Append sequences and targets
                sequences.append(seq_x)
                targets.append(seq_y)
                vessel_ids.append(vessel_id)
                shipping_ids.append(shipping_id)
                times.append(last_seq_time)

        previus_future_target = future_target
        future_target = int(future_target**(1.05))
        if future_target == previus_future_target:
            future_target += 1
        print(future_target)

    # Convert lists to numpy arrays
    X = np.array(sequences).astype(np.float32)
    y = np.array(targets).astype(np.float32)
    vessel_ids = np.array(vessel_ids)
    shipping_ids=np.array(shipping_ids)
    times = np.array(times)
    
    return X, y, vessel_ids, shipping_ids

In [ ]:
import os

print(train_data_engineered.columns)
train_data_sequenced_X = []
train_data_sequenced_Y = []

if os.path.exists(
    f"intermediate/train_data_sequenced_X_{sequence_length}_{max_future_target}.npy"
):
    print("Loading sequenced data from file")
    train_data_sequenced_X = np.load(
        f"intermediate/train_data_sequenced_X_{sequence_length}_{max_future_target}.npy",
        allow_pickle=True,
    )
    train_data_sequenced_Y = np.load(
        f"intermediate/train_data_sequenced_Y_{sequence_length}_{max_future_target}.npy",
        allow_pickle=True,
    )
else:
    print("Creating sequenced data")
    train_data_sequenced_X, train_data_sequenced_Y, vessel_ids, shipping_ids = create_sequences(
        train_data_engineered,
        sequence_length=sequence_length,
        train_instances_per_vessel=train_instances_per_vessel,
    )

    # Transform the IDs to integer labels
    vessel_ids = vessel_encoder.transform(vessel_ids)
    shipping_ids = shipping_encoder.transform(shipping_ids)
    
    #np.save(f"intermediate/train_data_sequenced_X_{sequence_length}_{future_target}.npy",train_data_sequenced_X)
    #np.save(f"intermediate/train_data_sequenced_Y_{sequence_length}_{future_target}.npy",train_data_sequenced_Y)

# train_data_shifted_df = train_data_shifted_df.drop(columns=["time"], axis=1)

Index(['time', 'vesselId', 'shippingLineId', 'seconds_to_eta',
       'port_latitude_sin', 'port_latitude_cos', 'port_longitude_sin',
       'port_longitude_cos', 'cog_sog_sin', 'cog_sog_cos', 'latitude_sin',
       'latitude_cos', 'longitude_sin', 'longitude_cos'],
      dtype='object')
Creating sequenced data
Feature Columns: ['seconds_to_eta', 'port_latitude_sin', 'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos', 'cog_sog_sin', 'cog_sog_cos', 'latitude_sin', 'latitude_cos', 'longitude_sin', 'longitude_cos', 'time_diff']
2
3
4
5
6
7
8
9
10
11
12
13
14
15
17
19
22
25
29
34
40
48
58
71
87
108
136
173
223
292
387
521


In [19]:

def append_last_known_data_test(
    test_data: pd.DataFrame,
    known_data: pd.DataFrame,
     vessel_information: pd.DataFrame
):
    """
    Groups training data by vesselId and propagates all data from the last known location.
    Pads sequences shorter than sequence_length.

    Args:
        test_data (pd.DataFrame): The test data containing 'vesselId' and 'time'.
        known_data (pd.DataFrame): The known data to extract last known positions.

    Returns:
        Tuple[np.ndarray, pd.Series]: A tuple containing the numpy array of sequences and the original times.
    """
    vessel_information=vessel_information.set_index("vesselId")
    
    test_data["time"] = pd.to_datetime(test_data["time"])

     # Determine the feature columns (excluding 'time' and 'vesselId')
    
    # Combine non_sequential_feature_columns with other columns to exclude them from sequential_feature_columns
    sequential_feature_columns = known_data.columns.values
    
    
    sequential_known_data=known_data[sequential_feature_columns]

    # Group known_data by 'vesselId', sort by 'time', and take the last sequence_length records
    grouped_data = (
        sequential_known_data.sort_values("time")
        .groupby("vesselId")
        .tail(sequence_length)
        .reset_index(drop=True)
    )

    grouped_data["time_diff"] = (
        (grouped_data.groupby("vesselId")["time"]
        .diff(-1)
        .dt.total_seconds()/10**6)
        .abs()
        .fillna(0)
    )

    test_data_numpy = []
    vessel_ids=[]
    shipping_ids=[]
    all_time_diffs = []


    for idx, row in test_data.iterrows():
        vessel_id = row["vesselId"]
        current_vessel_data=vessel_information.loc[vessel_id]
        shipping_id=current_vessel_data["shippingLineId"]        
        
        test_time = row["time"]

        # Extract current data for the current vessel_id
        current_data = grouped_data[grouped_data["vesselId"] == vessel_id]
        
        if current_data.empty:
            raise ValueError(f"vesselId {vessel_id} not found in known_data.")

        # Get the actual sequence length
        actual_sequence_length = len(current_data)

        # Calculate cumulative time difference and scale it
        last_idx = current_data.index[-1]
        time_diff_cumulative = (test_time - current_data.loc[last_idx, "time"]).total_seconds()/10**6

        # Update 'time_diff' in vessel_data
        current_data.loc[last_idx, "time_diff"] = time_diff_cumulative

        # Select wanted sequential features
        current_data = current_data.drop(["time","vesselId","shippingLineId"],axis=1)

        # Handle padding if the sequence is shorter than sequence_length
        if actual_sequence_length < sequence_length:
            # Calculate how many padding steps are needed
            padding_needed = sequence_length - actual_sequence_length

            # Create padding array with zeros (or a specified padding value)
            padding_array = np.zeros((padding_needed, current_data.shape[1]))

            # Optionally, you can set specific values for 'time_diff' in the padding
            time_diff_index = current_data.columns.get_loc("time_diff")
            padding_array[:, time_diff_index] = 0  # Set 'time_diff' in padding to zero

            # Convert vessel_data to numpy array
            vessel_array = current_data.values

            # Concatenate padding to the beginning of the sequence
            vessel_array = np.vstack((padding_array, vessel_array))
        else:
            # Convert vessel_data to numpy array
            vessel_array = current_data.values

        # Ensure the final sequence has the correct length
        assert vessel_array.shape[0] == sequence_length, f"Sequence length mismatch for vessel {vessel_id}"

        # Append to the list
        test_data_numpy.append(vessel_array)
        vessel_ids.append(vessel_id)
        shipping_ids.append(shipping_id)
        
    original_time = test_data["time"]
    vessels = test_data["vesselId"]
    

    return np.array(test_data_numpy).astype(np.float32), np.array(vessel_ids), np.array(shipping_ids)


In [21]:
test_data_X=0

if os.path.exists(
    f"intermediate/test_data_sequenced_X_{sequence_length}_{max_future_target}.npy"
    ):
    print("loading test data")
    test_data_X=np.load(f"intermediate/test_data_sequenced_X_{sequence_length}_{max_future_target}.npy", allow_pickle=True)
else:
    print("creating test data")
    test_data_X, test_vessel_ids,test_shipping_ids = append_last_known_data_test(test_data, train_data_engineered,vessel_data.copy())
    
    test_vessel_ids = vessel_encoder.transform(test_vessel_ids)
    test_shipping_ids = shipping_encoder.transform(test_shipping_ids)
    #np.save(f"intermediate/test_data_sequenced_X_{sequence_length}_{max_future_target}.npy", test_data_X)

creating test data


In [22]:
print(test_data_X[-2,-1,:])
print(train_data_sequenced_X[9,-1,:])
print(test_data_X.shape)

[0.26093897 0.8085266  0.5884597  0.18856698 0.9820603  0.4152345
 0.43379942 0.85954654 0.51105744 0.37444007 0.9272511  0.448658  ]
[0.263523   0.32460994 0.945848   0.95571744 0.2942859  0.43754008
 0.5479032  0.13979493 0.9901805  0.9739797  0.22663516 0.00258   ]
(51739, 10, 12)


In [23]:
@tf.keras.utils.register_keras_serializable()
class ContinuousPositionalEncoding(layers.Layer):
    def __init__(self, d_model, **kwargs):
        """
        Initializes the ContinuousPositionalEncoding layer.

        Args:
            d_model (int): The dimensionality of the model.
            **kwargs: Additional keyword arguments for the Layer base class.
        """
        super(ContinuousPositionalEncoding, self).__init__(**kwargs)
        self.d_model = d_model

    @tf.function
    def call(self, time_differences):
        """
        Computes the positional encoding based on cumulative time differences.

        Args:
            time_differences (tf.Tensor): A tensor of shape (batch_size, sequence_length)
                                          containing cumulative time differences.

        Returns:
            tf.Tensor: Positional encoding tensor of shape (batch_size, sequence_length, d_model).
        """
        # Compute cumulative time positions
        time_positions = tf.cumsum(time_differences, axis=1)*sequence_length # Shape: (batch_size, sequence_length)
        position = tf.expand_dims(time_positions, axis=-1)    # Shape: (batch_size, sequence_length, 1)

        # Compute the angle rates
        i = tf.range(self.d_model, dtype=tf.float32)          # Shape: (d_model,)
        i = tf.reshape(i, (1, 1, self.d_model))               # Shape: (1, 1, d_model)
        angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(self.d_model, tf.float32))  # Shape: (1, 1, d_model)
        angle_rads = position * angle_rates                   # Shape: (batch_size, sequence_length, d_model)

        # Apply sin to even indices and cos to odd indices
        sines = tf.sin(angle_rads[..., 0::2])                 # Shape: (batch_size, sequence_length, d_model/2)
        cosines = tf.cos(angle_rads[..., 1::2])               # Shape: (batch_size, sequence_length, d_model/2)

        # Concatenate sines and cosines along the last axis
        pos_encoding = tf.concat([sines, cosines], axis=-1)   # Shape: (batch_size, sequence_length, d_model)

        return pos_encoding



In [24]:

@tf.keras.utils.register_keras_serializable()
class LearnedPositionalEncoding(layers.Layer):
    def __init__(self, sequence_length, d_model, **kwargs):
        super(LearnedPositionalEncoding, self).__init__(**kwargs)
        self.pos_embedding = layers.Embedding(input_dim=sequence_length, output_dim=d_model)

    def call(self, inputs):
        """
        Adds learned positional embeddings to the input tensor.

        Args:
            inputs (tf.Tensor): Input tensor of shape (batch_size, sequence_length, d_model).

        Returns:
            tf.Tensor: Tensor with positional embeddings added.
        """
        batch_size = tf.shape(inputs)[0]
        sequence_length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=sequence_length, delta=1)
        pos_embeddings = self.pos_embedding(positions)  # Shape: (sequence_length, d_model)
        pos_embeddings = tf.expand_dims(pos_embeddings, axis=0)  # Shape: (1, sequence_length, d_model)
        pos_embeddings = tf.tile(pos_embeddings, [batch_size, 1, 1])  # Shape: (batch_size, sequence_length, d_model)
        return inputs + pos_embeddings


In [25]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Layer Normalization
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    # Multi-Head Attention
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs  # Residual Connection

    # Feed-Forward Network
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)
    return x + res  # Residual Connection


In [26]:
from tensorflow import keras
from tensorflow.keras import layers

def build_transformer_model(
    sequence_length,
    num_features,
    num_targets,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    d_model,
    num_vessels,
    vessel_embedding_dim,
    num_shipping_lines,
    shipping_embedding_dim,
    dropout=0,
    mlp_dropout=0,
):
    # Define the input layer for sequential features
    feature_inputs = keras.Input(shape=(sequence_length, num_features), name='feature_inputs')
    
    # Apply masking to the input features
    masked_inputs = layers.Masking(mask_value=0.0)(feature_inputs)

    # Project features to d_model dimensions
    x = layers.Dense(d_model)(masked_inputs)  # Shape: (batch_size, sequence_length, d_model)

    # Apply learned positional encoding
    x = LearnedPositionalEncoding(sequence_length, d_model)(x)
    
    # Transformer Encoder Blocks
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # Global Average Pooling for sequence dimension reduction
    x = layers.GlobalAveragePooling1D()(x)

    # Vessel ID Input and Embedding
    vessel_id_input = keras.Input(shape=(1,), name='vessel_id_input')
    vessel_embedding = layers.Embedding(input_dim=num_vessels, output_dim=vessel_embedding_dim, name='vessel_embedding')(vessel_id_input)
    vessel_embedding = layers.Flatten()(vessel_embedding)  # Shape: (batch_size, vessel_embedding_dim)
    
    # Shipping ID Input and Embedding
    shipping_id_input = keras.Input(shape=(1,), name='shipping_id_input')
    shipping_embedding = layers.Embedding(input_dim=num_shipping_lines, output_dim=shipping_embedding_dim, name='shipping_embedding')(shipping_id_input)
    shipping_embedding = layers.Flatten()(shipping_embedding)  # Shape: (batch_size, shipping_embedding_dim)

    # Concatenate All Paths
    concatenated = layers.concatenate([x, vessel_embedding, shipping_embedding], axis=-1)  # Shape: (batch_size, d_model + vessel_embedding_dim + shipping_embedding_dim + 32)

    # MLP for Regression
    for units in mlp_units:
        concatenated = layers.Dense(units, activation="relu")(concatenated)
        concatenated = layers.Dropout(mlp_dropout)(concatenated)

    # Output layer for regression targets
    outputs = layers.Dense(num_targets)(concatenated)
    
    # Define the model
    model = keras.Model(inputs=[feature_inputs, vessel_id_input, shipping_id_input], outputs=outputs)
    return model




In [27]:
def get_aggregated_time_diff(data):
    aggregated_time_diff=data[:,:,-1]
    
    # Step 2: Shift time_diff_next by one position to create time_diff_previous
    # Set the first entry in each sequence to 0 to represent no previous time difference
    aggregated_time_diff = np.roll(aggregated_time_diff, shift=1, axis=1)
    aggregated_time_diff[:, 0] = 0  # Set the first entry of each sequence to 0

    # Step 3: Calculate the cumulative sum of time_diff_previous along each sequence
    aggregated_time_diff = np.cumsum(aggregated_time_diff, axis=1)  # Shape (1471269, sequence_length)
    return(aggregated_time_diff)

In [29]:
d_model=16

model = build_transformer_model(
    sequence_length=sequence_length,
    num_features=num_features,
    num_targets=num_targets,  # Define this variable based on your task
    head_size=int(d_model/4),
    num_heads=4,
    ff_dim=d_model*4,
    num_transformer_blocks=4,
    mlp_units=[d_model*4],
    d_model=d_model,
    num_vessels=750,
    vessel_embedding_dim=16,
    num_shipping_lines=30,
    shipping_embedding_dim=4,
    dropout=0.1,
    mlp_dropout=0.1,
)


model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(),
    metrics=["mae"],
)

model.summary()

/lhome/kristsey/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'learned_positional_encoding_1' (of type LearnedPositionalEncoding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ feature_inputs      │ (None, 10, 12)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_1 (Masking) │ (None, 10, 12)    │          0 │ feature_inputs[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 10, 16)    │        208 │ masking_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ learned_positional… │ (None, 10, 16)    │        160 │ dense_1[0][0]     │
│ (LearnedPositional… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 16)    │         32 │ learned_position… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 16)    │      1,088 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 10, 16)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 10, 16)    │          0 │ dropout_1[0][0],  │
│                     │                   │            │ learned_position… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 16)    │         32 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 10, 64)    │      1,088 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 10, 64)    │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 10, 16)    │      1,040 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 10, 16)    │          0 │ dense_3[0][0],    │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 16)    │         32 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 16)    │      1,088 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 10, 16)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 10, 16)    │          0 │ dropout_4[0][0],  │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 16)    │         32 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 10, 64)    │      1,088 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 28,236 (110.30 KB)

 Trainable params: 28,236 (110.30 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from sklearn.model_selection import train_test_split
import gc

train_data_sequenced_X=train_data_sequenced_X.astype(np.float32)
train_data_sequenced_Y=train_data_sequenced_Y.astype(np.float32)

X_train, X_val, y_train, y_val, vessels_train, vessels_val, shipping_ids_train, shipping_ids_val = train_test_split(
    train_data_sequenced_X,             # Sequential input features
    train_data_sequenced_Y,             # Target output
    vessel_ids,                            # Vessel IDs
    shipping_ids,                       # Shipping IDs
    test_size=0.1,
    random_state=42
)

del train_data_sequenced_X
gc.collect() 

In [31]:
def create_dataset(sequential_features,vessel_ids, shipping_ids, targets, batch_size):
    # Create the dataset
    dataset = tf.data.Dataset.from_tensor_slices(((sequential_features,vessel_ids,shipping_ids), targets))
    
    # Shuffle, batch, and prefetch in one pipeline
    dataset = dataset.shuffle(buffer_size=10000) \
                     .batch(batch_size) \
                     .prefetch(tf.data.AUTOTUNE)
    return dataset

# Create training and validation datasets using the optimized function
train_dataset = create_dataset(X_train,vessels_train,shipping_ids_train, y_train, batch_size=1024)
val_dataset = create_dataset(X_val,vessels_val,shipping_ids_val, y_val, batch_size=1024)


In [ ]:

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored metric
)

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='training_checkpoint/{epoch:02d}.keras',  # Filepath to save the model
    save_freq=25,
    save_best_only=False  # Save the model at the specified frequency, not just the best model
)

# Assuming model, train_dataset, and val_dataset are already defined
# Train the model with the EarlyStopping callback
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[early_stopping,checkpoint]
)

Epoch 1/50


/lhome/kristsey/.local/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['feature_inputs', 'vessel_id_input', 'shipping_id_input']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


    1/18050 ━━━━━━━━━━━━━━━━━━━━ 49:16:56 10s/step - loss: 0.9048 - mae: 0.7261

2024-11-08 23:55:38.669506: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


18050/18050 ━━━━━━━━━━━━━━━━━━━━ 736s 40ms/step - loss: 0.0215 - mae: 0.0874 - val_loss: 0.0040 - val_mae: 0.0337
Epoch 2/50
18050/18050 ━━━━━━━━━━━━━━━━━━━━ 728s 40ms/step - loss: 0.0083 - mae: 0.0582 - val_loss: 0.0034 - val_mae: 0.0308
Epoch 3/50
18050/18050 ━━━━━━━━━━━━━━━━━━━━ 748s 41ms/step - loss: 0.0078 - mae: 0.0564 - val_loss: 0.0032 - val_mae: 0.0301
Epoch 4/50
18050/18050 ━━━━━━━━━━━━━━━━━━━━ 601s 33ms/step - loss: 0.0075 - mae: 0.0554 - val_loss: 0.0030 - val_mae: 0.0292
Epoch 5/50
13616/18050 ━━━━━━━━━━━━━━━━━━━━ 2:22 32ms/step - loss: 0.0073 - mae: 0.0550

In [ ]:
# Predict future positions
prediction_x=[test_data_X,test_vessel_ids,test_shipping_ids]
print(test_vessel_ids.dtype)
predictions = model.predict(prediction_x)

predictions=np.array(predictions).astype("float32")
print(predictions.dtype)

lat_predictions_radians = np.arctan2(predictions[:,0], predictions[:,1])
long_predictions_radians = np.arctan2(predictions[:,2],predictions[:,3])

# Convert radians to degrees
lat_predictions_degrees = np.rad2deg(lat_predictions_radians)
long_predictions_degrees = np.rad2deg(long_predictions_radians)



In [ ]:
print(lat_predictions_degrees)
print(long_predictions_degrees)

In [ ]:
predictions = pd.DataFrame({
        'ID': range(len(lat_predictions_degrees)),
        'longitude_predicted': long_predictions_degrees,
        'latitude_predicted': lat_predictions_degrees
    })
if pd.isna(predictions["latitude_predicted"]).any():
    print("oh no")
print(predictions.columns)

In [ ]:
predictions.to_csv("predictions_transformer.csv", index=False, float_format='%.15f')

In [ ]:
print(len(predictions))
print(len(test_data))
print(predictions["longitude_predicted"])